In [59]:
from scipy import linalg as la
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from scipy.optimize import fmin_cg
import time
%matplotlib inline

## Least Mean Squares

In [2]:
# Loading Continuous data set
batch = 100
data = datasets.load_boston()
X = data.data
Y = data.target

#making batch
X_test = train_test_split(X, Y, test_size = batch)[1]
Y_test = train_test_split(X, Y, test_size = batch)[3]

##### Implemented Linear Regression

In [3]:
class LR():
    '''Linear Regression Class
        Functions:  .pad - inserts a columns of ones to data set
                    .fit - fitting the model according to the given training data
                    .predict - predict class labels for samples in X
    '''
    def pad(self, X):
        ''' Accepts: X - matrix, data
            Returns: X - matrix, data, with a column of ones
        '''
        n = X.shape[0]
        X = np.hstack((np.ones(n).reshape(n,1), X))
        return X
    
    def fit(self, X, Y):
        ''' Accepts: X - matrix, data
                     Y - vector, label
            Returns: w - vector, weights
        '''
        N = len(Y)
        X = self.pad(X)
        self.w = la.solve(np.dot(np.transpose(X), X), np.dot(np.transpose(X),Y))
        SSE = np.sum((Y - np.dot(X, self.w))**2)
        N = len(Y)
        self.sigma = SSE/N
        return self.w
        
    def predict(self, X):
        '''Accepts: X - matrix, data
           Returns: X.dot(w)
        '''
        return np.dot(self.pad(X), self.w)

In [52]:
class LMS():
    '''Least Mean Squares
        Functions:  .pad - inserts a columns of ones to data set
                    .fit - fitting the model according to the given training data
                    .predict - predict class labels for samples in X
    '''
    def pad(self, X):
        ''' Accepts: X - matrix, data
            Returns: X - matrix, data, with a column of ones
        '''
        n = X.shape[0]
        X = np.hstack((np.ones(n).reshape(n,1), X))
        return X
    
    def fit(self, X, Y, tol, iters):
        ''' Accepts: X - matrix, data
                     Y - vector, label
                     tol - float
                     iters - int 
            Returns: w - vector, weights
        '''
        X = self.pad(X)
        N, D  = X.shape
        self.w = np.ones(D)
        for i in range(N):
            DQ = np.dot(X[i, :].T, np.dot(X[i, :],self.w) - Y[i])
            eta_0 = 1./(1+i)
            DQ = DQ/la.norm(DQ)
            w_1 = (self.w - eta_0*DQ)
            self.w = w_1
        return self.w
        
    def predict(self, X):
        '''Accepts: X - matrix, data
           Returns: X.dot(w)
        '''
        return np.dot(self.pad(X), self.w)

##### Timing Linear Regression and Least Mean Squares

In [53]:
#Timing Implented Logistic Regression
begin = time.time()
m = LR()
print m.fit(X_test, Y_test)
print m.predict(X[0:3,:])
end = time.time()
diff = end-begin
print diff

#Timing Implemented Least Mean Squares
begin1 = time.time()
model = LMS()
print model.fit(X_test,Y_test, .1, 100)
print model.predict(X[0:3,:])
end1 = time.time()
diff1 = end1-begin1
print diff1

[  3.53312439e+00   2.00991937e+00   9.10052719e-02  -1.72940740e-01
  -6.05507295e-03   3.75747451e+01   1.95548435e+01   1.74767622e+00
  -3.52950123e+01  -7.59694578e+00  -6.44377709e+01   7.34092710e-02
   2.91538972e-01   6.71728521e-01  -3.74363463e-02   1.04173285e+02
  -1.02647144e+01   2.44995255e+01  -1.47259842e+02  -6.65924796e+00
  -2.64835219e+02  -8.51979261e-01  -6.32137233e-02  -2.70945385e-02
   7.28786724e-03  -1.28215134e+01  -6.20424348e-01  -6.10193333e-01
   2.18022979e+01   3.15350066e+00  -8.40394245e+00]
[-0.38341243 -0.01652866 -0.52502355]
0.00500011444092
[ 0.99586768  0.961372    0.92389559  0.74683281 -0.06480871  0.99958209
  0.99935921  0.99943655  0.99983767  0.99909315  0.99966766  0.99845574
  0.99332005  0.98805739  0.92149864  0.99996215  0.99980434  0.99970238
  0.99992021  0.9998743   0.99997461  0.95744164  0.9043894   0.71691192
 -0.25489775  0.99946496  0.9987423   0.99859943  0.99960336  0.9987117
  0.99959108]
[-132.06245903 -214.5092016  -1

## Perceptron Algorithm

In [47]:
#Loading Linearly Separable Data
batch = 100
data = datasets.load_breast_cancer()
X = data.data
Y = data.target
X_test = train_test_split(X, Y, test_size = batch)[1]
Y_test = train_test_split(X, Y, test_size = batch)[3]

#Changing 0's in Y to -1's
Y_test1 = Y_test.copy()
for i in range(len(Y_test)):
    if Y_test[i] == 0:
        Y_test[i] = -1

### Logistic Regression

In [44]:
def logistic_reg(Lambda, X, Y):
    '''Logistic Regression

        Parameters: Lambda - postive float
                    X      - matrix, data
                    Y      - vector, binary
        Returns:    w_hat  - vector, coefficient 
                            of the features in the 
                            decision function
    '''
    n = X.shape[0]
    X = np.hstack((np.ones(n).reshape(n,1), X))
    N, D = X.shape

    def sigm(w, x):
        '''Sigmoid function
            Parameters: w      - vector, weights
                        x      - vector, data
            Returns:    1/(1+e^(w^Tx))
        '''
        return 1./(1 + np.exp(-x.dot(w)))

    def F(w):
        '''Negative Log Likelihood Function
            Parameters: w      - vector, initial guess
            Returns:    w_hat  - vector, coefficient of 
                                the features in the 
                                decision functioon
        '''
        return np.sum(Y *np.log(1+np.exp(-X.dot(w))) + (1-Y)* np.log(np.exp(X.dot(w))+1))

    #Using Newton's Method to find w_hat
    w_hat = fmin_cg(F, np.zeros(D), disp = False)
    
    return w_hat

### Perceptron Algorithm

In [62]:
def perceptron(X, Y):
    '''Perceptron Algorithm
        Accepts: X - matrix, data
                 Y - vector, label
        Returns: w_0 - vector, weights
    '''
    n = X.shape[0]
    X = np.hstack((np.ones(n).reshape(n,1), X))
    N, D  = X.shape
    w_0 = np.ones(D)
    for i in range(0, N):
        eta = 1./(1+i)
        y_hat = np.sign(np.dot(w_0, X[i, :]))
        if y_hat != Y[i]:
            w_0 = w_0 + Y[i]*X[i,:]*eta
    return w_0

##### Predict Function

In [63]:
def predict(x, w):
    '''Predict function for logistic regression (a.k.a. bias)
        Parameters: x - vector, data
                    w - vector, w_hat, or coefficients of the 
                        features of the decision function
        Returns:    k - vector of true and false values
    '''
    def sigm(w, x):
        '''Sigmoid function
            Parameters: w      - vector, weights
                        x      - vector, data
            Returns:    1/(1+e^(w^Tx))
        '''
        return 1./(1 + np.exp(-x.dot(w)))
    n = x.shape[0]
    x = np.hstack((np.ones(n).reshape(n,1), x))
    k = sigm(w, x)
    return k >= .5

##### Timing Perceptron with Logisitic Regression

In [65]:
#Timing Implemented Perceptron
begin1 = time.time()
w_hat = perceptron(X_test, Y_test)
p = predict(X_test, w_hat)
end1 = time.time()
diff1 = end1 - begin1
print "Time it take to fit perceptron is \t\t\t" + str(diff1)

#Timing Implemented Logistic Regression
begin = time.time()
w_hat_lr = logistic_reg(0, X_test, Y_test1)
p1 = predict(X_test, w_hat_lr)
end = time.time()
diff = end- begin
print "Time it take to fit for Logisitic Regression Solver is \t" + str(diff)




Time it take to fit perceptron is 			0.00200009346008
Time it take to fit for Logisitic Regression Solver is 	0.233000040054


C:\Users\clmeha\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: overflow encountered in exp
C:\Users\clmeha\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: RuntimeWarning: overflow encountered in exp
C:\Users\clmeha\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: RuntimeWarning: invalid value encountered in multiply
